In [1]:
import gzip
import logging
from pathlib import Path
from typing import List, Optional

import pandas as pd
from rdkit import Chem
from tqdm import tqdm
from joblib import Parallel, delayed


from rxitect.chem import mol_utils
from rxitect.structs.vocabulary import SmilesVocabulary

logger = logging.getLogger(__name__)
MIN_TOKEN_LEN = 10
MAX_TOKEN_LEN = 100
N_CHEMBL_COMPOUNDS = {
    "v30": 2_157_379
}


def generate_smiles_corpus(
    raw_data_filepath: Path,
    destdir: Path,
    corpus_type: str,
    requires_clean: bool = True,
    is_isomeric: bool = False,
    create_voc_file: bool = False,
):
    """
    This method constructs a dataset with molecules represented as SMILES. Each molecule will be decomposed
    into a series of tokens. In the end, all the tokens will be put into one set as vocabulary.

    Args:
        raw_data_filepath (Path): The file path of input, either .sdf file or tab-delimited file
        destdir (Path): The file path to write the created files to.
        corpus_type (str): type of the file to be processed and written. Can be 'chembl' or 'ligand'.
        requires_clean (bool): If the molecule is required to be clean, the charge metal will be
                removed and only the largest fragment will be kept.
        is_isomeric (bool): If the molecules in the dataset keep conformational information. If not,
                the conformational tokens (e.g. @@, @, \, /) will be removed. False by default.
        create_voc_file:
    """
    if corpus_type == "chembl":
        df = _get_mols_from_sdf(is_isomeric, raw_data_filepath)
    elif corpus_type == "ligand":
        df = pd.read_table(raw_data_filepath).Smiles.dropna()
    else:
        raise ValueError("Only valid corpus types are 'chembl' and 'ligand'.")

    voc = SmilesVocabulary(vocabulary_file_path=None)
    words = set()
    canons = []
    tokens = []
    if requires_clean:
        smiles = set()
        for smile in tqdm(df, desc="Cleaning molecules"):
            try:
                smile = mol_utils.clean_mol(smile, is_isomeric=is_isomeric)
                smiles.add(Chem.CanonSmiles(smile))
            except Exception as e:
                logger.warning("Parsing Error: ", e)
    else:
        smiles = df.values
    for smile in tqdm(smiles, desc="Tokenizing SMILES"):
        token = voc.tokenize(smile)
        # Only collect the organic molecules
        if {"C", "c"}.isdisjoint(token):
            logger.warning("Non-organic token detected: ", smile)
            continue
        # Remove the metal tokens
        if not {"[Na]", "[Zn]"}.isdisjoint(token):
            logger.warning("Metal token detected: ", smile)
            continue
        # control the minimum and maximum of sequence length.
        if MIN_TOKEN_LEN < len(token) <= MAX_TOKEN_LEN:
            words.update(token)
            canons.append(smile)
            tokens.append(" ".join(token))

    if create_voc_file:
        # output the vocabulary file
        with open(destdir / f"smiles_voc.txt", "w") as voc_file:
            voc_file.write("\n".join(sorted(words)))

    outfile = destdir / f"smiles_{corpus_type}_corpus.txt"
    _write_corpus(canon_smiles=canons, outfile=outfile, tokens=tokens)


def _write_corpus(canon_smiles, outfile, tokens):
    """Output the dataset file as tab-delimited file"""
    corpus_df = pd.DataFrame()
    corpus_df["smiles"] = canon_smiles
    corpus_df["token"] = tokens
    corpus_df.drop_duplicates(subset="smiles")
    corpus_df.to_csv(path_or_buf=outfile, sep="\t", index=False)


def _get_mols_from_sdf(is_isomeric: bool, raw_data_filepath: Path, chembl_version: Optional[str] = "v30") -> List[str]:
    """Handle sdf file with RDkit"""
    inf = gzip.open(raw_data_filepath)
    fsuppl = Chem.ForwardSDMolSupplier(inf)
    smiles = []
    total_mols = N_CHEMBL_COMPOUNDS.get(chembl_version) if chembl_version else None
    for mol in tqdm(fsuppl, total=total_mols, desc="Processing ChEMBL molecules"):
        try:
            smiles.append(Chem.MolToSmiles(mol, is_isomeric))
        except Exception as e:
            logger.warning(f"Was not able to convert {mol} to smiles: {e}")
    return smiles


/home/julius/miniconda3/envs/rxt-dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = _get_mols_from_sdf(raw_data_filepath="../data/raw/chembl_30.sdf.gz", is_isomeric=False)

Processing ChEMBL molecules:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 2136187/2157379 [09:31<00:05, 3739.12it/s]


In [3]:
# 2_136_187

In [6]:
open("../data/processed/chembl_30_smiles.txt", "w").write('\n'.join(smi for smi in df))

115060425

In [10]:
%%time
smiles = set()
for smile in tqdm(df[:1000], desc="Cleaning molecules"):
    try:
        smile = mol_utils.clean_mol(smile)
        smiles.add(Chem.CanonSmiles(smile))
    except Exception as e:
        print("Parsing Error: ", e)

Cleaning molecules: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 320.27it/s]

CPU times: user 3.12 s, sys: 12 ms, total: 3.13 s
Wall time: 3.12 s


In [12]:
df_set = set(df[:1000])
for smile in tqdm(df_set, desc="Cleaning molecules"):
    try:
        smile = mol_utils.clean_mol(smile)
        smiles.add(Chem.CanonSmiles(smile))
    except Exception as e:
        print("Parsing Error: ", e)

Cleaning molecules: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:03<00:00, 324.89it/s]


In [30]:
%%time
smiles = set()
for smile in df[:1000]:
    try:
        smile = mol_utils.clean_mol(smile)
        smiles.add(Chem.CanonSmiles(smile))
    except Exception as e:
        print("Parsing Error: ", e)

CPU times: user 3.02 s, sys: 0 ns, total: 3.02 s
Wall time: 3.02 s


In [24]:
def clean_and_canonalize(smiles: str) -> str:
    processed_smiles = None
    try:
        smiles = mol_utils.clean_mol(smiles)
        processed_smiles = Chem.CanonSmiles(smiles)
    except Exception as e:
        print("Parsing Error: ", e)
    return processed_smiles

In [33]:
%%time
clean_smiles = set(Parallel(n_jobs=-1)(delayed(clean_and_canonalize)(smi) for smi in df[:1000]))

CPU times: user 426 ms, sys: 208 ms, total: 634 ms
Wall time: 908 ms


In [28]:
len(clean_smiles)

992

In [29]:
%%time
clean_smiles = set((clean_and_canonalize(smi) for smi in df[:1000]))

CPU times: user 3.02 s, sys: 0 ns, total: 3.02 s
Wall time: 3.02 s


In [34]:
sv = SmilesVocabulary()

In [37]:
tks = [sv.tokenize(cs) for cs in clean_smiles]

In [43]:
%%time

words = set()
canon_smiles = []
tokenized_smiles = []
for smi in tqdm(df, desc="Tokenizing SMILES"):
    tokens = sv.tokenize(smi)
    # Only collect the organic molecules
    if {"C", "c"}.isdisjoint(tokens):
        print("Non-organic token detected: ", smi)
        continue
    # Remove the metal tokens
    if not {"[Na]", "[Zn]"}.isdisjoint(tokens):
        print("Metal token detected: ", smi)
        continue
    # control the minimum and maximum of sequence length.
    if 10 < len(tokens) <= 100:
        words.update(tokens)
        canon_smiles.append(smi)
        tokenized_smiles.append(" ".join(tokens))


Tokenizing SMILES:   2%|██▎                                                                                                                     | 41903/2136187 [00:00<00:09, 212092.24it/s]

Non-organic token detected:  O=S(=O)(O)NS(=O)(=O)O
Non-organic token detected:  OB(O)O
Non-organic token detected:  F[Ag]


Tokenizing SMILES:   4%|████▉                                                                                                                   | 86810/2136187 [00:00<00:09, 220585.43it/s]

Non-organic token detected:  O=O.[O-][Cl+]O.[O-][Cl+]O.[O-][Cl+]O.[O-][Cl+]O
Non-organic token detected:  NS(=O)(=O)O
Non-organic token detected:  [Cl-].[Li+]
Non-organic token detected:  OO
Non-organic token detected:  [N-]=[N+]=[N-]
Non-organic token detected:  [N-]=[N+]=[N-].[Na+]
Non-organic token detected:  O=N[O-].[Na+]
Non-organic token detected:  NN


Tokenizing SMILES:   6%|███████▎                                                                                                               | 130811/2136187 [00:00<00:09, 218615.17it/s]

...

Non-organic token detected:  [Ag+].[C-]#N.[C-]#N.[Na+]
Non-organic token detected:  F[As-](F)(F)(F)(F)F.[Na+]
Non-organic token detected:  F[Al-3](F)(F)(F)(F)F.[Na+].[Na+].[Na+]
Non-organic token detected:  [O-2].[Zn+2]
Non-organic token detected:  O.O.O.O.O.O.O.O=S(=O)([O-])[O-].[Mg+2]
Non-organic token detected:  O=S([O-])([O-])=S.[K+].[K+].[K]S[K]
Non-organic token detected:  O.O.O.O.O.O=P([O-])([O-])[O-].O=P([O-])([O-])[O-].[Mg+2].[Mg+2].[Mg+2]
Non-organic token detected:  O=[Mg].O=[Mg].O=[Mg].O=[Si]=O.O=[Si]=O.O=[Si]=O.O=[Si]=O
Non-organic token detected:  O=S(O)(O)=S.[K]S[K]


Tokenizing SMILES:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 2089526/2136187 [00:09<00:00, 196790.18it/s]

Non-organic token detected:  S.S.S
Metal token detected:  Nc1c(N=Nc2ccc(-c3ccccc3OCCC[Na])nc2)cc(S(=O)(=O)[O-])c2ccccc12.[Na+]
Non-organic token detected:  OP(O)(=S)S


Tokenizing SMILES: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2136187/2136187 [00:10<00:00, 210032.95it/s]

Non-organic token detected:  [Ba]
Non-organic token detected:  [Ra]
Non-organic token detected:  [Rb]
Non-organic token detected:  Br
Non-organic token detected:  F[AsH](F)(F)(F)(F)F
Non-organic token detected:  [Cs]
Non-organic token detected:  I
CPU times: user 10.2 s, sys: 116 ms, total: 10.3 s
Wall time: 10.3 s


In [45]:
%%time
smiles = set(Parallel(n_jobs=-1)(delayed(mol_utils.clean_and_canonalize)(smi) for smi in df))

AttributeError: module 'rxitect.chem.mol_utils' has no attribute 'clean_and_canonalize'